# Cashflow Generation - Part 1

This is first part in a series that will examine how OpenRedukti facilities can be used to generate cashflows for trades.

Cashflow generation is a complex task especially when you consider a real trade specified in FpML. There are many options that need to be considered. OpenRedukti does not yet support all possible options, as its focus is on trades that are cleared by CCPs, which forms a subset of all trades that can be specified in FpML.

### Fixed Legs

Let us first look at how fixed legs are handled.

OpenRedukti represents fixed legs as a series of simple cashflows where each cashflow has a known amount and payment date. This means that you need to precompute the amount for each cashflow, using information such as `notional`, `fixed rate` and `day count fraction`. Here is how you do it.

Lets suppose that we are creating cashflows for a two year swap. The fixed leg pays every 6 months. Lets define some dates first.

In [48]:
import redukti
from redukti import enums_pb2 as enums

In [50]:
start_date = redukti.Date.dmy(8,7,2019) # 8 July 2019
end_date = redukti.Date.dmy(8,7,2021)   # 8 July 2021
notional = 1000000                      # 1 million
currency = enums.EUR
fixed_rate = 0.01                       # 1%

### ScheduleGenerator

OpenRedukti provides a ScheduleGenerator we can use to generate the schedule. Typically we do this for each leg of the swap.

The Schedule Generator takes a parameter of type `schedule_pb2.ScheduleParameters`. Lets create that and add relevant values.

In [51]:
from redukti import schedule_pb2 as schedule

fixed_schedule_parameters = schedule.ScheduleParameters()
fixed_schedule_parameters.effective_date = start_date.serial()
fixed_schedule_parameters.termination_date = end_date.serial()

Above we created the `fixed_schedule_parameters` object and set the start and end dates. Note that we need to set the date values to the serial number. Also note that the two dates must be unadjusted.

Next lets let some other required attributes. Since we will be constructing a EUR swap, we need to use the `EUTA` calendar for schedule generation.

In [52]:
fixed_schedule_parameters.payment_calendars.append(enums.EUTA)
fixed_schedule_parameters.payment_frequency = enums.TENOR_12M
fixed_schedule_parameters.period_convention = enums.MODIFIED_FOLLOWING

I think those are all the fields we need. Lets review what we have in the `fixed_schedule_parameters` object.

In [53]:
fixed_schedule_parameters

effective_date: 43654
termination_date: 44385
payment_frequency: TENOR_12M
period_convention: MODIFIED_FOLLOWING
payment_calendars: EUTA

We can now try to generate the schedule. We do this bay calling a static method in `redukti.ScheduleGenerator` class.

In [54]:
fixed_schedule = redukti.ScheduleGenerator.generate_schedule(fixed_schedule_parameters)

Okay, we can now look at the results.

In [55]:
fixed_schedule

adjusted_start_dates: 43654
adjusted_start_dates: 44020
adjusted_end_dates: 44020
adjusted_end_dates: 44385
adjusted_payment_dates: 44020
adjusted_payment_dates: 44385

So we obtained three lists in the structure.

* adjusted_start_dates
* adjusted_end_dates
* adjusted_payment_dates

The date values are all in serial number form.

We can use this information to construct the cashflows for the fixed leg. As mentioned before, in order to so this we need to use notional, day count fraction, and fixed rate.

### Converting a schedule to cashflows

Our goal is to construct a `cashflow_pb2.CFCollection` object.
This object will contain a list of `cashflow_pb2.Stream` objects.
There will be two streams, one for fixed leg, and the second for floating leg.

We have to follow the Google Protobuf convention to create these objects.

In [56]:
from redukti import cashflow_pb2

In [57]:
cfcollection = cashflow_pb2.CFCollection()
fixed_stream = cfcollection.streams.add()
floating_stream = cfcollection.streams.add()

Now we need to populate fixed_stream.

In [58]:
day_fraction = redukti.DayFraction(enums.ACT_360)

for i in range(0, len(fixed_schedule.adjusted_payment_dates)):
    adjusted_start = redukti.Date(fixed_schedule.adjusted_start_dates[i])
    adjusted_end   = redukti.Date(fixed_schedule.adjusted_end_dates[i])
    year_fraction  = day_fraction.year_fraction(adjusted_start, adjusted_end)
    amount         = notional * fixed_rate * year_fraction
    # Create the cashflow
    single = fixed_stream.cashflows.add()                # Add a single payment coupon
    single.simple.currency = currency                    # The simple subobject represents a fixed flow of known amount
    single.simple.amount   = amount
    single.simple.payment_date = fixed_schedule.adjusted_payment_dates[i]
    single.simple.discounting_index_family = enums.EONIA # We want to use EONIA curve for discounting

So now we should have the fixed cashflow stream! Lets have a look.

In [59]:
fixed_stream

cashflows {
  simple {
    currency: EUR
    amount: 10166.666666666666
    payment_date: 44020
    discounting_index_family: EONIA
  }
}
cashflows {
  simple {
    currency: EUR
    amount: 10138.888888888889
    payment_date: 44385
    discounting_index_family: EONIA
  }
}

### The Floating Stream

For our floating stream, we will complicate things a bit by asking for compounding annually. That is, the floating rate will be computed every 6 months and compounded with one annual payment. So this means that we will have two calculation periods of 6 months inside each payment period.

In [60]:
float_schedule_parameters = schedule.ScheduleParameters()
float_schedule_parameters.effective_date = start_date.serial()
float_schedule_parameters.termination_date = end_date.serial()
float_schedule_parameters.payment_calendars.append(enums.EUTA)
float_schedule_parameters.payment_frequency = enums.TENOR_12M
float_schedule_parameters.payment_convention = enums.MODIFIED_FOLLOWING
float_schedule_parameters.calculation_frequency = enums.TENOR_6M
float_schedule_parameters.period_convention = enums.MODIFIED_FOLLOWING
float_schedule_parameters.period_calendars.append(enums.EUTA)

Notice the additional parameters that specify that we want calculations to occur every 6 months. However Payment will occur annually. Now lets see what this produces:

In [61]:
float_schedule = redukti.ScheduleGenerator.generate_schedule(float_schedule_parameters)
float_schedule

adjusted_start_dates: 43654
adjusted_start_dates: 43838
adjusted_start_dates: 44020
adjusted_start_dates: 44204
adjusted_end_dates: 43838
adjusted_end_dates: 44020
adjusted_end_dates: 44204
adjusted_end_dates: 44385
adjusted_payment_dates: 0
adjusted_payment_dates: 44020
adjusted_payment_dates: 0
adjusted_payment_dates: 44385

The schedule looks similar to that we produced for the fixed leg except that every alternate payment date is 0. The value of 0 means there is no payment in that period.

### Generating Floating cashflows

The steps for producing floating cashflows is somewhat more complex. Additionally we have to take care of compunding cashflow too.

In [62]:
coupon = None
for i in range(0, len(float_schedule.adjusted_payment_dates)):
    if not coupon:
        coupon = floating_stream.cashflows.add()
    calc_period = coupon.floating.floating_periods.add()
    calc_period.notional = notional
    calc_period.accrual_start_date = float_schedule.adjusted_start_dates[i]
    calc_period.accrual_end_date   = float_schedule.adjusted_end_dates[i]
    calc_period.index = enums.EUR_EURIBOR_Reuters
    calc_period.tenor = enums.TENOR_6M
    calc_period.spread = 0.0
    if float_schedule.adjusted_payment_dates[i] != 0:
        # This is a payment period
        coupon.floating.currency = currency
        coupon.floating.day_count_fraction = enums.ACT_360
        coupon.floating.payment_date = float_schedule.adjusted_payment_dates[i]
        coupon.floating.compounding_method = enums.FLAT
        coupon.floating.discounting_index_family = enums.EONIA
        coupon = None # So next iteration creates a new coupon

Some things to note above.

* We create a coupon or payment period every alternate period. To make this generic we reset the coupon to `None` at the  beginning and after every payment. This triggers creation of new coupon at the next iteration of the loop
* In floating coupons, we supply the raw information such as notional, start and end dates, day count fraction. Additionally at the coupon level we provide the floating index and tenor to use.
* We specify that the compounding method should be `FLAT`. The other alternative that is supported is `STRAIGHT`.
* Finally as in the fixed stream, we specify that the discounting should use `EONIA` index family.

So now lets look at what was generated

In [64]:
floating_stream

cashflows {
  floating {
    currency: EUR
    floating_periods {
      notional: 1000000.0
      accrual_start_date: 43654
      accrual_end_date: 43838
      index: EUR_EURIBOR_Reuters
      tenor: TENOR_6M
    }
    floating_periods {
      notional: 1000000.0
      accrual_start_date: 43838
      accrual_end_date: 44020
      index: EUR_EURIBOR_Reuters
      tenor: TENOR_6M
    }
    compounding_method: FLAT
    day_count_fraction: ACT_360
    payment_date: 44020
    discounting_index_family: EONIA
  }
}
cashflows {
  floating {
    currency: EUR
    floating_periods {
      notional: 1000000.0
      accrual_start_date: 44020
      accrual_end_date: 44204
      index: EUR_EURIBOR_Reuters
      tenor: TENOR_6M
    }
    floating_periods {
      notional: 1000000.0
      accrual_start_date: 44204
      accrual_end_date: 44385
      index: EUR_EURIBOR_Reuters
      tenor: TENOR_6M
    }
    compounding_method: FLAT
    day_count_fraction: ACT_360
    payment_date: 44385
    discountin

We are almost ready with our cashflows. We need to decide which leg is paying and which one is receiving. Suppose the fixed leg is paying and float leg is receiving. We can indicate this as follows.

In [65]:
fixed_stream.factor = -1.0
floating_stream.factor = 1.0

I think we are done creating our cashflows. Let have a look at the whole.

In [66]:
cfcollection

streams {
  cashflows {
    simple {
      currency: EUR
      amount: 10166.666666666666
      payment_date: 44020
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 10138.888888888889
      payment_date: 44385
      discounting_index_family: EONIA
    }
  }
  factor: -1.0
}
streams {
  cashflows {
    floating {
      currency: EUR
      floating_periods {
        notional: 1000000.0
        accrual_start_date: 43654
        accrual_end_date: 43838
        index: EUR_EURIBOR_Reuters
        tenor: TENOR_6M
      }
      floating_periods {
        notional: 1000000.0
        accrual_start_date: 43838
        accrual_end_date: 44020
        index: EUR_EURIBOR_Reuters
        tenor: TENOR_6M
      }
      compounding_method: FLAT
      day_count_fraction: ACT_360
      payment_date: 44020
      discounting_index_family: EONIA
    }
  }
  cashflows {
    floating {
      currency: EUR
      floating_periods {
        notional: 100